# In this notebook we reproduce the experimental analysis on artificial data

### import necessary libraries and define functions for our experiments

In [1]:
from scheduling_functions import *
from scheduling_algorithms import *
from numpy import std
import numpy as np
import sys
import copy
from random import sample, randint, seed, random
from math import isclose, ceil, floor, e, sqrt
from statistics import mean
from decimal import *
from fractions import *
import matplotlib.pyplot as plt
from operator import add
import dill

In [2]:
#creates a bounded random walk:

def random_walk_creation(num_jobs, step_size, random_seed, m, M):
    seed(random_seed)

    ws = [0]*num_jobs
    ws[0] = randint(m,M)
    steps = [randint(-step_size,step_size) for i in range(1,num_jobs)]
    for i in range(1, num_jobs):
        ws[i] = ws[i-1] + steps[i-1]
        ws[i] = min(ws[i], M)
        ws[i] = max(ws[i], m)
    return ws

In [3]:
#creates a job instance given a list of weights and T

def job_instance_creation(ws, D):
    # dictionary: key --> job id
    #            value --> (weight, release time , deadline)
    J = {}
    job_id = 1
    i = 0
    for job_weight in ws:
        J[job_id] = (job_weight , i, i+D)
        i+=1
        job_id+=1
    return J

In [4]:
def LAS_energy_ratio(_J_true, _J_pred, epsilon, alpha, dt):
    
    #compute energy of LAS algorithm
    J_true = copy.deepcopy(_J_true)
    J_pred = copy.deepcopy(_J_pred)
    
    speed_sol = LAS(J_pred, J_true, epsilon, dt, alpha)
    energy_LAS = sum([s**alpha for s in speed_sol])*dt
    
   
    #compute speedlist and energu consumption of the optimal schedule of the true instance
    J_true = copy.deepcopy(_J_true)
    J_pred = copy.deepcopy(_J_pred)
    optimal_alg_speed_list, _ = Optimal_Alg(J_true)
    energy_optimal = compute_energy(optimal_alg_speed_list, alpha)
    
    return float(energy_LAS)/energy_optimal    

In [5]:
#returns the energy ratio AVR_energy/Optimal_energy

def AVR_energy_ratio(_J, alpha):
    
    J = copy.deepcopy(_J)
    #speed list of average rate
    AVR_speed_list = Avg_rate(J)
    #energy consumption of AVR
    energy_AVR = compute_energy(AVR_speed_list, alpha)
    
    J = copy.deepcopy(_J)
    #speed list of the optimal schedule
    optimal_alg_speed_list, _ = Optimal_Alg(J)
    #energy consumption of the optimal schedule
    energy_optimal = compute_energy(optimal_alg_speed_list, alpha)  
    
    return float(energy_AVR)/energy_optimal    

In [6]:
#returns the energy ratio OA_energy/Optimal_energy

def OA_energy_ratio(_J, alpha):
    
    J = copy.deepcopy(_J)
    #speed list of Optimal Available
    OA_speed_list = OptimalOnline(J)
    #energy consumption of Optimal Available
    energy_OA = sum([s**alpha for s in OA_speed_list])
    
    J = copy.deepcopy(_J)
    #speed list of the optimal schedule
    optimal_alg_speed_list, _ = Optimal_Alg(J)
    #energy consumption of the optimal schedule
    energy_optimal = compute_energy(optimal_alg_speed_list, alpha)    
   
    return float(energy_OA)/energy_optimal

In [7]:
#returns the energy ratio BKP_energy/Optimal_energy

def BKP_energy_ratio(_J, granularity, alpha):
    
    J = copy.deepcopy(_J)
    #energy consumption of the BKP algorithm
    energy_BKP = BKP_alg(J, granularity, alpha)

    J = copy.deepcopy(_J)
    #speed list of the optimal schedule
    optimal_alg_speed_list, _ = Optimal_Alg(J)
    #energy consumption of the optimal schedule
    energy_optimal = compute_energy(optimal_alg_speed_list, alpha)  
    
    return float(energy_BKP)/energy_optimal    

### we set the parameters of the experiments

In [8]:
# instance length and T
num_jobs = 200
D = 20

num_of_experiments = 20


step_size = 5
M = 80
m = 20

# alpha parameter of the energy consumption
alpha = 3

# time granularity for BKP algorithm
BKP_granularity = 0.25

# granularity of CONV algorithm
dt = 0.01

# robustness parameters epsilon which will be tested
epsilons=[Fraction(1,100), Fraction(20,100), Fraction(40,100), Fraction(60,100), Fraction(80,100)]


### to increase reproducibility we perform experiments on the same set of (random) true instances with fixed seeds

In [9]:
J_true_lst = []
w_true_lst = []
for j in range(0,num_of_experiments):
     #create a random walk
    w_true = random_walk_creation(num_jobs, step_size, random_seed=j, M= M, m= m)
    w_true_lst.append(w_true)
    #job instance creation
    J_true = job_instance_creation(w_true, D)
    
    J_true_lst.append(J_true)

### Online Algorithms tested

(1) Average Rate Heuristic (AVR)

(2) Optimal Available (OA)

(3) Bansal, Kimbrel and Pruhs algorithm (BKP)


In [10]:
y_AVR = []
y_BKP = []
y_OA = []
dummy_y_AVR = []
dummy_y_BKP = []
dummy_y_OA = []
for j in range(0,num_of_experiments):
    J_true = J_true_lst[j]
    
    AVR = AVR_energy_ratio(J_true,alpha)
    BKP = BKP_energy_ratio(J_true,BKP_granularity, alpha)
    OA = OA_energy_ratio(J_true, alpha)
    dummy_y_AVR.append(AVR)
    dummy_y_BKP.append(BKP)
    dummy_y_OA.append(OA)
std_AVR = std(dummy_y_AVR)
std_BKP = std(dummy_y_BKP)
std_OA  = std(dummy_y_OA)
y_AVR.append(mean(dummy_y_AVR))
y_BKP.append(mean(dummy_y_BKP))
y_OA.append(mean(dummy_y_OA))

print("AVR: ", y_AVR[-1])
print("Std ", std_AVR)
print("=====================")   
print("BKP: ", y_BKP[-1])
print("Std ", std_BKP)
print("=====================")
print("OA: ", y_OA[-1])
print("Std ", std_OA)
print("=====================")

AVR:  1.2675809010639774
Std  0.07104477119543437
BKP:  7.8803797060825955
Std  1.2663495212504527
OA:  1.1985253933487527
Std  0.07003246274593491


In [11]:
print("worst AVR: ", max(dummy_y_AVR))
print("=====================")   
print("worst BKP: ", max(dummy_y_BKP))
print("=====================")
print("worst OA: ", max(dummy_y_OA))
print("=====================")

worst AVR:  1.3827228085885481
worst BKP:  10.380889582716827
worst OA:  1.3613134092905024


### (1) Accurate predictor

#### We create the artificial predictions of our "Accurate predictor"

In [12]:
J_pred_lst = []
for j in range(0,num_of_experiments):
    w_true = w_true_lst[j]
    
    seed(j)
    error = [randint(-step_size, step_size) for _ in range(0,num_jobs)]
    
    w_pred = list(map(add, w_true, error))    
    #jon instance creation
    J_pred = job_instance_creation(w_pred, D)
    
    J_pred_lst.append(J_pred)


#### We test the performance of the Learning Augmented Scheduling (LAS) algorithm when combined with an "Accurate predictor"

In [13]:
for epsilon in epsilons:
    print("EPSILON = ", epsilon)
    y_LAS_scheduling = []
    
    dummy_y_LAS_schedulling = []
    for j in range(0,num_of_experiments):
        J_true = J_true_lst[j]
        w_true = w_true_lst[j]
        J_pred = J_pred_lst[j]
            
        
        LAS_scheduling = LAS_energy_ratio(J_true, J_pred, epsilon, alpha, dt)

        dummy_y_LAS_schedulling.append(LAS_scheduling)
    

    
    y_LAS_scheduling.append(mean(dummy_y_LAS_schedulling))
    std_LAS_scheduling = std(dummy_y_LAS_schedulling)
    
    
    print("LAS scheduling: ", y_LAS_scheduling[-1])
    print("Std of LAS scheduling ", std_LAS_scheduling)
    print("=====================")     

EPSILON =  1/100
LAS scheduling:  1.0075397494928156
Std of LAS scheduling  0.00432815821742801
EPSILON =  1/5
LAS scheduling:  1.0130706593352214
Std of LAS scheduling  0.005007302937615069
EPSILON =  2/5
LAS scheduling:  1.017974594981769
Std of LAS scheduling  0.00578540691382172
EPSILON =  3/5
LAS scheduling:  1.0222734294264288
Std of LAS scheduling  0.0065593000867124
EPSILON =  4/5
LAS scheduling:  1.0262250101527997
Std of LAS scheduling  0.007335483925709539


### (2) Random predictor

#### we create the artificial predictions of our "Random predictor"

In [14]:
J_pred_lst = []
for j in range(0,num_of_experiments):
    seed(j)
    error = [randint(-step_size, step_size) for _ in range(0,num_jobs)]
    
    w_pred = [randint(m,M) for _ in range(0,num_jobs)]
    
    
    #jon instance creation
    J_pred = job_instance_creation(w_pred, D)
    
    J_pred_lst.append(J_pred)

#### We test the performance of the Learning Augmented Scheduling (LAS) algorithm when combined with a "Random predictor"

In [15]:
for epsilon in epsilons:
    print("EPSILON = ", epsilon)
    y_LAS_scheduling = []
    
    dummy_y_LAS_schedulling = []
    for j in range(0,num_of_experiments):
        J_true = J_true_lst[j]
        J_pred = J_pred_lst[j]
            
        
        LAS_scheduling = LAS_energy_ratio(J_true, J_pred, epsilon, alpha, dt)

        dummy_y_LAS_schedulling.append(LAS_scheduling)
    
    
    y_LAS_scheduling.append(mean(dummy_y_LAS_schedulling))
    std_LAS_scheduling = std(dummy_y_LAS_schedulling)
    
    
    print("LAS scheduling: ", y_LAS_scheduling[-1])
    print("Std of LAS scheduling ", std_LAS_scheduling)
    print("=====================")   

EPSILON =  1/100
LAS scheduling:  1.2394752538463858
Std of LAS scheduling  0.17250843763873255
EPSILON =  1/5
LAS scheduling:  1.2239815495837538
Std of LAS scheduling  0.14724187216333076
EPSILON =  2/5
LAS scheduling:  1.2127922268228106
Std of LAS scheduling  0.12990693420691735
EPSILON =  3/5
LAS scheduling:  1.2065392431134532
Std of LAS scheduling  0.1197132996358433
EPSILON =  4/5
LAS scheduling:  1.2033045200807153
Std of LAS scheduling  0.11307804236368527


### Misleading predictor

#### We create the artificial predictions of our "Misleading predictor"

In [16]:
J_pred_lst = []
for j in range(0,num_of_experiments):
    w_true = w_true_lst[j]
    
    w_pred = []
    for i in range(0,num_jobs):
        w_pred.append((M-w_true[i]) + m)
    
    
    #jon instance creation
    J_pred = job_instance_creation(w_pred, D)
    
    J_pred_lst.append(J_pred)

#### We test the performance of the Learning Augmented Scheduling (LAS) algorithm when combined with a "Misleading predictor"

In [17]:
for epsilon in epsilons:
    print("EPSILON = ", epsilon)
    y_LAS_scheduling = []
    
    dummy_y_LAS_schedulling = []
    for j in range(0,num_of_experiments):
        J_true = J_true_lst[j]
        J_pred = J_pred_lst[j]
            
        
        LAS_scheduling = LAS_energy_ratio(J_true, J_pred, epsilon, alpha, dt)

        dummy_y_LAS_schedulling.append(LAS_scheduling)
    
    
    y_LAS_scheduling.append(mean(dummy_y_LAS_schedulling))
    std_LAS_scheduling = std(dummy_y_LAS_schedulling)
    
    
    print("LAS scheduling: ", y_LAS_scheduling[-1])
    print("Std of LAS scheduling ", std_LAS_scheduling)
    print("=====================")     

EPSILON =  1/100
LAS scheduling:  1.4040283929377526
Std of LAS scheduling  0.18755807607768799
EPSILON =  1/5
LAS scheduling:  1.4062699075727747
Std of LAS scheduling  0.1857250485744134
EPSILON =  2/5
LAS scheduling:  1.4069707494536996
Std of LAS scheduling  0.1839259349874737
EPSILON =  3/5
LAS scheduling:  1.406920096057823
Std of LAS scheduling  0.18142601778836964
EPSILON =  4/5
LAS scheduling:  1.4072313575386666
Std of LAS scheduling  0.17917555303510502


In [18]:
for epsilon in epsilons:
    print("EPSILON = ", epsilon)
    y_LAS_scheduling = []
    
    dummy_y_LAS_schedulling = []
    for j in range(0,num_of_experiments):
        J_true = J_true_lst[j]
        J_pred = J_pred_lst[j]
            
        
        LAS_scheduling = LAS_energy_ratio(J_true, J_pred, epsilon, alpha, dt)

        dummy_y_LAS_schedulling.append(LAS_scheduling)
    
    
    y_LAS_scheduling.append(max(dummy_y_LAS_schedulling))
    
    
    print("worst LAS scheduling: ", y_LAS_scheduling[-1])
    print("=====================")     

EPSILON =  1/100
worst LAS scheduling:  1.766393940546647
EPSILON =  1/5
worst LAS scheduling:  1.7694663182826558
EPSILON =  2/5
worst LAS scheduling:  1.767065435654167
EPSILON =  3/5
worst LAS scheduling:  1.7578459665461634
EPSILON =  4/5
worst LAS scheduling:  1.750487346257285
